In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from statsmodels.tsa.stattools import adfuller

In [3]:
# Import data
df = pd.read_csv('prices.txt', engine='python', sep='   ', header=None, names=[f"stock{i}" for i in range(50)])
df

,stock0,stock1,stock2,stock3,stock4,stock5,stock6,stock7,stock8,stock9,...,stock40,stock41,stock42,stock43,stock44,stock45,stock46,stock47,stock48,stock49
0,13.46,71.65,48.46,50.52,52.10,13.00,18.98,47.71,69.49,49.96,...,32.64,55.76,14.46,58.94,36.71,52.62,49.33,36.22,49.00,56.09
1,13.48,72.10,48.52,50.50,52.06,12.95,18.95,47.84,69.73,49.93,...,32.52,55.97,14.44,59.81,36.64,52.58,49.20,36.27,48.84,56.08
2,13.47,72.35,48.48,50.62,51.80,12.79,18.98,47.98,69.60,49.33,...,32.48,56.34,14.50,59.04,36.89,52.49,49.48,36.39,48.56,55.90
3,13.53,72.51,48.42,50.75,51.66,12.66,18.96,48.74,69.54,49.67,...,32.59,56.32,14.40,58.73,36.94,52.40,49.42,36.41,49.00,56.14
4,13.64,71.99,48.40,50.65,51.97,12.62,18.89,48.88,69.68,49.46,...,32.64,56.32,14.36,59.01,37.03,52.44,49.79,36.42,48.14,55.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,10.32,63.28,45.54,43.35,52.64,7.91,17.30,55.30,67.83,58.73,...,28.81,79.78,10.85,61.76,30.12,48.51,82.09,34.95,35.80,52.57
746,10.32,63.34,45.56,43.28,52.73,7.87,17.26,54.92,67.95,58.62,...,28.78,80.46,10.77,61.49,29.85,48.40,81.43,34.99,35.58,53.10
747,10.32,63.23,45.55,43.25,52.66,7.83,17.32,54.67,67.94,59.15,...,28.73,81.15,10.75,60.36,29.77,48.41,81.90,35.00,35.45,53.21
748,10.20,63.12,45.56,43.19,52.51,7.71,17.33,55.18,67.99,59.64,...,28.73,82.22,10.81,59.24,29.61,48.42,81.33,35.19,35.57,53.01


In [23]:
def adftest(stock_name):
    print(f"##### ADF Test for {stock_name} #####")

    prices = df[stock_name].iloc[-100:]
    returns = np.log1p(prices.pct_change())
    returns = returns[~np.isnan(returns)]

    print('Returns')
    test = adfuller(returns)
    dfoutput = pd.Series(
        test[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in test[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)
    print()

for stock_name in df.columns:
    adftest(stock_name)

##### ADF Test for stock0 #####
Returns
Test Statistic                -9.149109e+00
p-value                        2.716832e-15
#Lags Used                     0.000000e+00
Number of Observations Used    9.800000e+01
Critical Value (1%)           -3.498910e+00
Critical Value (5%)           -2.891516e+00
Critical Value (10%)          -2.582760e+00
dtype: float64

##### ADF Test for stock1 #####
Returns
Test Statistic                -1.000819e+01
p-value                        1.807875e-17
#Lags Used                     0.000000e+00
Number of Observations Used    9.800000e+01
Critical Value (1%)           -3.498910e+00
Critical Value (5%)           -2.891516e+00
Critical Value (10%)          -2.582760e+00
dtype: float64

##### ADF Test for stock2 #####
Returns
Test Statistic                 -5.205852
p-value                         0.000009
#Lags Used                      3.000000
Number of Observations Used    95.000000
Critical Value (1%)            -3.501137
Critical Value (5%)        

# Hurst Exponent

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from hurst import compute_Hc

def hurst(stock_name, window=-100):
    # Evaluate Hurst equation
    stock_prices = df[stock_name].iloc[window:]
    H, c, data = compute_Hc(stock_prices, kind='price', simplified=True)

    # print("H={:.4f}, c={:.4f}".format(H,c))
    return H, c

def interpret(H, threshold=0.3):
    if H < threshold:
        return('anti-persistent behaviour')
    if H > (1 - threshold):
        return('persistent behaviour')
    return('Brownian Motion')

results = [hurst(stock_name, -250) for stock_name in df.columns]
hurst_df = pd.DataFrame(results, columns=['H', 'c'])
hurst_df['behaviour'] = hurst_df.apply(lambda row: interpret(row['H'], 0.4), axis=1)
hurst_df

,H,c,behaviour
0,0.846762,0.564462,persistent behaviour
1,0.571684,1.283309,Brownian Motion
2,0.657656,0.876060,persistent behaviour
3,0.659046,0.828771,persistent behaviour
4,0.495511,1.188831,Brownian Motion
5,0.821999,0.565808,persistent behaviour
6,0.713595,0.812322,persistent behaviour
7,0.542719,1.179717,Brownian Motion
8,0.497729,1.296357,Brownian Motion
9,0.458486,1.474364,Brownian Motion
